In [2]:
#https://www.youtube.com/watch?v=Nt7WJa2iu0s
import requests 
from bs4 import BeautifulSoup
import pandas as pd
from io import StringIO


In [3]:
standigs_url = 'https://fbref.com/en/comps/9/PRemier-League-Stats'
epl_data = requests.get(standigs_url)
soup = BeautifulSoup(epl_data.text)
stdgs_tbl = soup.select('table.stats_table')[0]
links = stdgs_tbl.find_all('a')
lnks = [l.get("href") for l in links]
team_urls = [f"https://fbref.com{l}" for l in lnks if '/squad' in l]
team_urls

IndexError: list index out of range

In [29]:
def team_stats_extract(url):
    season_df = pd.DataFrame()
    for team in url:        
        print(team)
        team_url = team
        team_nm = team_url.split('/')[-1].replace('-Stats', '')
        team_data = requests.get(team_url)
        matches = pd.read_html(StringIO(team_data.text), match = 'Scores & Fixtures')
        soupt = BeautifulSoup(team_data.text)
        linkst = [l.get("href") for l in soupt.find_all('a')]
        l = [f"https://fbref.com{l}" for l in linkst if l and  '/all_comps/shootin' in l]
        shots_data = requests.get(l[0])
        shooting = pd.read_html(StringIO(shots_data.text), match = 'Shooting')[0]
        shooting.columns = shooting.columns.droplevel()
        matches_df = matches[0].merge(shooting[["Date", "Comp", "Opponent", 'Sh', 'SoT', 'Dist', "FK", "PK", "PKatt" ]], on = ['Date', 'Comp', 'Opponent'], how = 'left')
        matches_df.columns = [col.upper() for col in matches_df.columns]

        print(matches_df.shape)
        matches_df.loc[:,'TEAM'] = team_nm
        season_df = pd.concat([season_df, matches_df], axis = 0)
    return season_df


In [30]:
PL24 = team_stats_extract(team_urls)

https://fbref.com/en/squads/18bb7c10/Arsenal-Stats


ValueError: No tables found